In [14]:
from datasets import load_dataset

dataset = load_dataset("yelp_review_full")
dataset["train"][100]

KeyboardInterrupt: 

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("google-bert/bert-base-uncased")


def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)


tokenized_datasets = dataset.map(tokenize_function, batched=True)

In [3]:
small_train_dataset = (
    tokenized_datasets["train"].shuffle(seed=42).select(range(1000))
)
small_eval_dataset = (
    tokenized_datasets["test"].shuffle(seed=42).select(range(1000))
)

In [ ]:
import adapters
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(
    "google-bert/bert-base-uncased", num_labels=5
)

adapters.init(model)

# model.add_adapter("my_lora_adapter", config="lora")
# model.train_adapter("my_lora_adapter")

from adapters import BnConfig

config = BnConfig(
    mh_adapter=True,
    output_adapter=True,
    reduction_factor=16,
    non_linearity="relu",
)
model.add_adapter("bottleneck_adapter", config=config)

model.train_adapter("bottleneck_adapter")

In [ ]:
model

In [ ]:
"bottleneck_adapter" in model.active_adapters

In [7]:
# model.freeze_model(False)

In [8]:
# from transformers import TrainingArguments

# training_args = TrainingArguments(output_dir=None)

In [9]:
import numpy as np
import evaluate

metric = evaluate.load("accuracy")

In [10]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [11]:
from transformers import TrainingArguments, Trainer

# training_args = TrainingArguments(output_dir="test_trainer", eval_strategy="epoch")
training_args = TrainingArguments(output_dir="./outputs")
# evaluation_strategy

In [12]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()